# Real-Time Optimization
## Modifier Adaptation with Bayesian Optimization using EIC acquisition
### Preliminary thesis results generation

In [14]:
# Loading the necessary packages
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import pandas as pd
import logging
from sklearn.utils import Bunch
logging.basicConfig(level=logging.ERROR)

from rto.models.williams_otto import WilliamsOttoReactor_CamelHump, WilliamsOttoReactor_Branin, WilliamsOttoReactorSimplified_CamelHump, WilliamsOttoReactorSimplified_Branin
from rto.optimization.optimizer import ModifierAdaptationOptimizer, ModelBasedOptimizer
from rto.optimization.bayesian import ModelBasedBayesianOptimizer
from rto.rto import RTOBayesian, RTO
from rto.adaptation.ma_gaussian_processes import MAGaussianProcesses
from rto.utils import generate_samples_uniform
from rto.experiment.analysis import ExperimentAnalyzer

# backup script
!python ../scripts/create_database.py -n thesis-analysis-03 -f /mnt/d/rto_data/
DATABASE = "/mnt/d/rto_data/thesis-analysis-03.db"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Creating database to /mnt/d/rto_data/thesis-analysis-03.db


In [15]:
# Our complete model will be called the "plant"
plant_branin = WilliamsOttoReactor_Branin()
plant_camelhump = WilliamsOttoReactor_CamelHump()

# And the uncertain is the "model"
model_branin = WilliamsOttoReactorSimplified_Branin()
model_camelhump = WilliamsOttoReactorSimplified_CamelHump()

# define the constraints
ubx = [5.5, 90]
lbx = [2.5, 70]
g_branin = np.array([0.16, 0.12])
g_camelhump = np.array([0.16, 0.12])

In [16]:
optimizer_branin = ModelBasedOptimizer(ub=ubx, lb=lbx, g=g_branin)
optimizer_camelhump = ModelBasedOptimizer(ub=ubx, lb=lbx, g=g_camelhump)

f_branin, u_branin ,_ = optimizer_branin.run(plant_branin, [])
f_camelhump, u_camelhump ,_ = optimizer_camelhump.run(model_branin, [])

f_branin_model, u_branin_model ,_ = optimizer_branin.run(model_branin, [])
f_camelhump_model, u_camelhump_model ,_ = optimizer_camelhump.run(model_camelhump, [])

print(f'Branin: u*={u_branin}, f*={f_branin}')
print(f'Camel Hump: u*={u_camelhump}, f*={f_camelhump}')

print(f'Branin (model): u*={u_branin_model}, f*={f_branin_model}')
print(f'Camel Hump (model): u*={u_camelhump_model}, f*={f_camelhump_model}')

Branin: u*=[ 4.39586571 70.88897484], f*=1.921573632520781
Camel Hump: u*=[ 4.73763516 85.9234244 ], f*=0.3979688565197037
Branin (model): u*=[ 4.74123808 85.93361563], f*=0.39789010666847524
Camel Hump (model): u*=[ 5.49924863 70.0035141 ], f*=-26.700747546803775


## Real-Time Optimization

## EIC acquisition function

### Branin

#### Optimizer Choice

In [17]:
# Define the system parameters
u_0 = u_branin_model
iterations = 30
initial_data_size = 5

# sample some initial data
u, y, measurements = generate_samples_uniform(model_branin, plant_branin, g_branin, u_0, initial_data_size, noise=0.0)
initial_data_branin = Bunch(u=u, y=y, measurements=measurements)

In [18]:
adaptation_bay_de = MAGaussianProcesses(model_branin, initial_data_branin, ub=ubx, lb=lbx, filter_data=True, neighbors_type='k_last')
adaptation_bay_sqp = MAGaussianProcesses(model_branin, initial_data_branin, ub=ubx, lb=lbx, filter_data=True, neighbors_type='k_last')

optimizer_bay_de = ModelBasedBayesianOptimizer(ub=ubx, lb=lbx, g=g_branin, solver={'name': 'de'}, backoff=0.0)
optimizer_bay_sqp = ModelBasedBayesianOptimizer(ub=ubx, lb=lbx, g=g_branin, solver={'name': 'sqp'}, backoff=0.0)

rto_bay_de = RTOBayesian(model_branin, plant_branin, optimizer_bay_de, adaptation_bay_de, iterations, db_file=DATABASE, name='MA-GP-Bayesian-DE-branin', noise=0.0)
rto_bay_sqp = RTOBayesian(model_branin, plant_branin, optimizer_bay_sqp, adaptation_bay_sqp, iterations, db_file=DATABASE, name='MA-GP-Bayesian-SQP-branin', noise=0.0)

u_0_feas_branin = initial_data_branin.u[-1]
rto_bay_sqp.run(u_0_feas_branin)
rto_bay_de.run(u_0_feas_branin)

2

#### Effect of noise

In [19]:
noise = 0.01
repetitions = 10

adaptation_de_noise = MAGaussianProcesses(model_branin, initial_data_branin, ub=ubx, lb=lbx, filter_data=True, neighbors_type='k_last')
adaptation_sqp_noise = MAGaussianProcesses(model_branin, initial_data_branin, ub=ubx, lb=lbx, filter_data=True, neighbors_type='k_last')

rto_ma_sqp_noise = RTO(model_branin, plant_branin, optimizer_bay_sqp, adaptation_sqp_noise, iterations, db_file=DATABASE, name='MA-GP-Bayesian-SQP-branin+noise', noise=noise)
rto_ma_de_noise = RTO(model_branin, plant_branin, optimizer_bay_de, adaptation_de_noise, iterations, db_file=DATABASE, name='MA-GP-Bayesian-DE-branin+noise', noise=noise)

for i in range(repetitions):
    rto_ma_sqp_noise.run(u_0_feas_branin)
    rto_ma_de_noise.run(u_0_feas_branin)

#### Different initial points with noise

In [20]:
# sample some initial data
# generate all the data before
n_datasets = 10
datasets_branin = []
for i in range(n_datasets):
    u_i, y_i, measurements_i = generate_samples_uniform(model_branin, plant_branin, g_branin, u_0, initial_data_size, noise=noise)
    datasets_branin.append(Bunch(u=u_i, y=y_i, measurements=measurements_i))

In [21]:
for i in range(n_datasets):
    initial_dataset = datasets_branin[i]
    adaptation_bay_de_noise = MAGaussianProcesses(model_branin, initial_dataset, ub=ubx, lb=lbx, filter_data=True, neighbors_type='k_last')
    rto_bay_de_noise = RTOBayesian(model_branin, plant_branin, optimizer_bay_de, adaptation_bay_de_noise, iterations, db_file=DATABASE, name='MA-GP-Bayesian-DE-branin+noise-datasets', noise=noise)
    rto_bay_de_noise.run(initial_dataset.u[-1])

### Camel Hump

In [22]:
# Define the system parameters
u_0 = u_camelhump_model
iterations = 30
initial_data_size = 5

# sample some initial data
u, y, measurements = generate_samples_uniform(model_camelhump, plant_camelhump, g_camelhump, u_0, initial_data_size, noise=0.0)
initial_data_camelhump = Bunch(u=u, y=y, measurements=measurements)

In [23]:
adaptation_bay_de = MAGaussianProcesses(model_camelhump, initial_data_camelhump, ub=ubx, lb=lbx, filter_data=True, neighbors_type='k_last')
adaptation_bay_sqp = MAGaussianProcesses(model_camelhump, initial_data_camelhump, ub=ubx, lb=lbx, filter_data=True, neighbors_type='k_last')

optimizer_bay_de = ModelBasedBayesianOptimizer(ub=ubx, lb=lbx, g=g_camelhump, solver={'name': 'de'}, backoff=0.0)
optimizer_bay_sqp = ModelBasedBayesianOptimizer(ub=ubx, lb=lbx, g=g_camelhump, solver={'name': 'sqp'}, backoff=0.0)

rto_bay_de = RTOBayesian(model_camelhump, plant_camelhump, optimizer_bay_de, adaptation_bay_de, iterations, db_file=DATABASE, name='MA-GP-Bayesian-DE-camelhump', noise=0.0)
rto_bay_sqp = RTOBayesian(model_camelhump, plant_camelhump, optimizer_bay_sqp, adaptation_bay_sqp, iterations, db_file=DATABASE, name='MA-GP-Bayesian-SQP-camelhump', noise=0.0)

u_0_feas_camelhump = initial_data_camelhump.u[-1]
rto_bay_sqp.run(u_0_feas_camelhump)
rto_bay_de.run(u_0_feas_camelhump)

34

#### Effect of noise

In [24]:
noise = 0.01
repetitions = 10

adaptation_de_noise = MAGaussianProcesses(model_camelhump, initial_data_camelhump, ub=ubx, lb=lbx, filter_data=True, neighbors_type='k_last')
adaptation_sqp_noise = MAGaussianProcesses(model_camelhump, initial_data_camelhump, ub=ubx, lb=lbx, filter_data=True, neighbors_type='k_last')

rto_ma_sqp_noise = RTO(model_camelhump, plant_camelhump, optimizer_bay_sqp, adaptation_sqp_noise, iterations, db_file=DATABASE, name='MA-GP-Bayesian-SQP-camelhump+noise', noise=noise)
rto_ma_de_noise = RTO(model_camelhump, plant_camelhump, optimizer_bay_de, adaptation_de_noise, iterations, db_file=DATABASE, name='MA-GP-Bayesian-DE-camelhump+noise', noise=noise)

for i in range(repetitions):
    rto_ma_sqp_noise.run(u_0_feas_camelhump)
    rto_ma_de_noise.run(u_0_feas_camelhump)

#### Different initial points with noise

In [26]:
# sample some initial data
# generate all the data before
n_datasets = 10
datasets_camelhump = []
for i in range(n_datasets):
    u_i, y_i, measurements_i = generate_samples_uniform(model_camelhump, plant_camelhump, g_camelhump, u_0, initial_data_size, noise=noise)
    datasets_camelhump.append(Bunch(u=u_i, y=y_i, measurements=measurements_i))

In [27]:
for i in range(n_datasets):
    initial_dataset = datasets_camelhump[i]
    adaptation_bay_de_noise = MAGaussianProcesses(model_camelhump, initial_dataset, ub=ubx, lb=lbx, filter_data=True, neighbors_type='k_last')
    rto_bay_de_noise = RTOBayesian(model_camelhump, plant_camelhump, optimizer_bay_de, adaptation_bay_de_noise, iterations, db_file=DATABASE, name='MA-GP-Bayesian-DE-camelhump+noise-datasets', noise=noise)
    rto_bay_de_noise.run(initial_dataset.u[-1])